In [4]:
from importers import SpikeImporter

# load the animal file
importer = SpikeImporter(filepath = "../data/11_10_27U2a_2030s_2100s_10000Hz.csv")

# try retrieving the APs and eletrical stimuli
print(importer.getActionPotentials(channel_name = "801 nw-1").iloc[range(0, 20)])
# print(importer.getElectricalStimuli(channel_name = "32 DigMark"))



        Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
943   0.0943     2.914           0     NaN     2.817
944   0.0944    -0.773           0     NaN    -0.865
945   0.0945   -20.635           0     NaN   -20.737
946   0.0946   -44.159           0     NaN   -44.353
947   0.0947    -9.466           0     NaN    -9.779
948   0.0948    18.031           0     NaN    17.711
949   0.0949    23.010           0     NaN    22.763
950   0.0950    26.093           0     NaN    25.919
951   0.0951    24.470           0     NaN    24.375
952   0.0952    13.260           0     NaN    13.238
953   0.0953     3.204           0     NaN     3.217
954   0.0954    -0.916           0     NaN    -0.990
3445  0.3445    -1.587           0  -1.587    -1.641
3446  0.3446    -4.354           0  -4.354    -4.414
3447  0.3447    -8.825           0  -8.825    -8.904
3448  0.3448   -31.555           0 -31.555   -31.669
3449  0.3449   -32.847           0 -32.847   -33.070
3450  0.3450    -1.821           0  -1.821    

In [5]:
from signal_artifacts import ActionPotential

# define the max. time so that two sequential waveform-values are belonging to the same AP
# the time is in seconds, so 0.005 equals 5ms
max_gap_time = 0.005
# state explicitly the name of the time column
time_column = "Time"

actpots = importer.getActionPotentials(channel_name = "801 nw-1")

# go through the dataframe and construct the APs
index = 0
while index < len(actpots.index) - 1:
    # assume that the first index is always the onset of an AP
    onset = index
    
    # increase the DF(!) index as long as the time distance to the next row is small enough
    while (abs(actpots.iloc[index + 1][time_column] - actpots.iloc[index][time_column]) < max_gap_time):
        index = index + 1
        
    # then, this is the last index
    offset = index + 1
        
    # create AP object from the shortened dataframe
    # print(str(onset) + " to " + str(offset))
    ap = ActionPotential(input_df = actpots.iloc[range(onset, offset)])
    
    # "jump" to the next AP
    index = index + 1

       Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
943  0.0943     2.914           0     NaN     2.817
944  0.0944    -0.773           0     NaN    -0.865
945  0.0945   -20.635           0     NaN   -20.737
946  0.0946   -44.159           0     NaN   -44.353
947  0.0947    -9.466           0     NaN    -9.779
948  0.0948    18.031           0     NaN    17.711
949  0.0949    23.010           0     NaN    22.763
950  0.0950    26.093           0     NaN    25.919
951  0.0951    24.470           0     NaN    24.375
952  0.0952    13.260           0     NaN    13.238
953  0.0953     3.204           0     NaN     3.217
954  0.0954    -0.916           0     NaN    -0.990
        Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
3445  0.3445    -1.587           0  -1.587    -1.641
3446  0.3446    -4.354           0  -4.354    -4.414
3447  0.3447    -8.825           0  -8.825    -8.904
3448  0.3448   -31.555           0 -31.555   -31.669
3449  0.3449   -32.847           0 -32.847   -33.070
3450  

         Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
30009  3.0009     1.836           0   1.836     1.851
30010  3.0010     3.510           0   3.510     3.532
30011  3.0011   -13.189           0 -13.189   -13.163
30012  3.0012   -41.438           0 -41.438   -41.472
30013  3.0013   -27.039           0 -27.039   -27.213
30014  3.0014     8.789           0   8.789     8.549
30015  3.0015    20.131           0  20.131    19.939
30016  3.0016    24.673           0  24.673    24.550
30017  3.0017    22.202           0  22.202    22.156
30018  3.0018    12.034           0  12.034    12.052
30019  3.0019     5.676           0   5.676     5.735
30020  3.0020     0.829           0   0.829     0.895
         Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
30094  3.0094    -2.075           0     NaN    -2.002
30095  3.0095    -2.182           0     NaN    -2.122
30096  3.0096   -10.096           0     NaN   -10.051
30097  3.0097   -37.247           0     NaN   -37.242
30098  3.0098   -38.366     

76013  7.6013     6.760           0   6.760     5.448
         Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
78504  7.8504    -0.270           0     NaN    -0.296
78505  7.8505    -2.945           0     NaN    -2.965
78506  7.8506   -20.106           0     NaN   -20.154
78507  7.8507   -45.659           0     NaN   -45.791
78508  7.8508   -26.123           0     NaN   -26.408
78509  7.8509     4.089           0     NaN     3.750
78510  7.8510    10.539           0     NaN    10.221
78511  7.8511    16.922           0     NaN    16.647
78512  7.8512    14.537           0     NaN    14.319
78513  7.8513     5.819           0     NaN     5.638
78514  7.8514     2.975           0     NaN     2.822
78515  7.8515     1.628           0     NaN     1.468
         Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
81006  8.1006     0.717           0   0.717     0.698
81007  8.1007    -2.716           0  -2.716    -2.741
81008  8.1008   -32.511           0 -32.511   -32.567
81009  8.1009   -37.547     

146072  14.6072     1.694           0   1.694     1.681
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
148564  14.8564     0.687           0   0.687     0.656
148565  14.8565    -4.054           0  -4.054    -4.092
148566  14.8566   -22.313           0 -22.313   -22.380
148567  14.8567   -41.580           0 -41.580   -41.718
148568  14.8568   -10.442           0 -10.442   -10.708
148569  14.8569    15.218           0  15.218    14.942
148570  14.8570    17.593           0  17.593    17.366
148571  14.8571    22.029           0  22.029    21.862
148572  14.8572    20.335           0  20.335    20.235
148573  14.8573     8.881           0   8.881     8.844
148574  14.8574     4.166           0   4.166     4.155
148575  14.8575     0.336           0   0.336     0.013
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
151066  15.1066    -4.684           0  -4.684    -4.704
151067  15.1067    -7.863           0  -7.863    -7.898
151068  15.1068   -16.225           0 -16.225   

221138  22.1138     2.365           0   2.365     2.763
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
223629  22.3629     4.405           0   4.405     4.504
223630  22.3630    -2.228           0  -2.228    -2.115
223631  22.3631   -28.264           0 -28.264   -28.168
223632  22.3632   -38.493           0 -38.493   -38.512
223633  22.3633    -4.242           0  -4.242    -4.378
223634  22.3634    14.999           0  14.999    14.869
223635  22.3635    21.189           0  21.189    21.123
223636  22.3636    24.612           0  24.612    24.617
223637  22.3637    18.677           0  18.677    18.767
223638  22.3638     6.480           0   6.480     6.617
223639  22.3639    -1.938           0  -1.938    -1.790
223640  22.3640    -1.450           0  -1.450    -0.487
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
226131  22.6131    -1.419           0  -1.419    -1.448
226132  22.6132    -3.983           0  -3.983    -4.007
226133  22.6133   -28.208           0 -28.208   

...

433823  43.3823     3.525           0   3.525     3.496
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
436314  43.6314     1.017           0   1.017     0.959
436315  43.6315    -0.244           0  -0.244    -0.278
436316  43.6316   -16.881           0 -16.881   -16.927
436317  43.6317   -38.610           0 -38.610   -38.737
436318  43.6318   -19.989           0 -19.989   -20.243
436319  43.6319     5.839           0   5.839     5.540
436320  43.6320    16.703           0  16.703    16.431
436321  43.6321    19.714           0  19.714    19.514
436322  43.6322    13.306           0  13.306    13.148
436323  43.6323     7.680           0   7.680     7.575
436324  43.6324     2.884           0   2.884     2.807
436325  43.6325    -1.038           0  -1.038    -2.933
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
438817  43.8817    -1.953           0  -1.953    -1.959
438818  43.8818    -7.573           0  -7.573    -7.600
438819  43.8819   -29.088           0 -29.088   

           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
503873  50.3873     0.346           0   0.346     0.291
503874  50.3874     3.866           0   3.866     3.816
503875  50.3875    -7.172           0  -7.172    -7.203
503876  50.3876   -35.589           0 -35.589   -35.675
503877  50.3877   -28.249           0 -28.249   -28.450
503878  50.3878     2.335           0   2.335     2.061
503879  50.3879    10.493           0  10.493    10.239
503880  50.3880    11.032           0  11.032    10.810
503881  50.3881     9.109           0   9.109     8.920
503882  50.3882    10.895           0  10.895    10.742
503883  50.3883    10.371           0  10.371    10.249
503884  50.3884     7.828           0   7.828     7.743
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
506375  50.6375    -1.689           0  -1.689    -1.834
506376  50.6376    -2.060           0  -2.060    -2.203
506377  50.6377    -9.369           0  -9.369    -9.509
506378  50.6378   -34.042           0 -34.042   

586457  58.6457    -1.999           0  -1.999    -1.407
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
588948  58.8948     0.788           0   0.788     0.761
588949  58.8949    -2.121           0  -2.121    -2.149
588950  58.8950   -23.727           0 -23.727   -23.777
588951  58.8951   -36.580           0 -36.580   -36.724
588952  58.8952    -5.585           0  -5.585    -5.829
588953  58.8953     9.964           0   9.964     9.707
588954  58.8954    16.474           0  16.474    16.270
588955  58.8955    22.034           0  22.034    21.867
588956  58.8956    17.817           0  17.817    17.739
588957  58.8957    10.788           0  10.788    10.761
588958  58.8958     3.082           0   3.082     3.077
588959  58.8959     3.021           0   3.021     4.154
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
591450  59.1450     0.636           0   0.636     0.794
591451  59.1451    -1.358           0  -1.358    -1.205
591452  59.1452   -15.457           0 -15.457   

659020  65.9020     3.555           0   3.555     3.526
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
661511  66.1511    -0.697           0  -0.697    -0.647
661512  66.1512     0.982           0   0.982     1.035
661513  66.1513    -7.858           0  -7.858    -7.822
661514  66.1514   -33.758           0 -33.758   -33.761
661515  66.1515   -19.506           0 -19.506   -19.621
661516  66.1516     6.393           0   6.393     6.232
661517  66.1517     9.664           0   9.664     9.530
661518  66.1518    13.092           0  13.092    12.988
661519  66.1519    13.504           0  13.504    13.450
661520  66.1520     7.573           0   7.573     7.555
661521  66.1521     3.937           0   3.937     3.938
661522  66.1522    -0.565           0  -0.565    -0.551
           Time  801 nw-1  32 DigMark  4 nw-1  1 Signal
664014  66.4014     0.707           0   0.707     0.557
664015  66.4015     1.740           0   1.740     1.596
664016  66.4016   -16.591           0 -16.591   

IndexError: single positional indexer is out-of-bounds